# Ideal Gas Simulation

### Recreation of Jeffrey Chang's project paper

https://jeffjar.me/files/simulating-ideal-gas.pdf

Molecular dynamics simulation involving the numerical solution to Newton's laws of motion.

Jeffrey Chang identifies three tenets of ideal gas theory which is demonstrated by the simulation:
1. Non-equilibrium states evolve to equilibrium;
2. The gas satisfies the equation of state $PV = Nk_BT$;
3. Fluctuations around equilibrium occur, and decrease with large N.

## Methodology

### Description:
- N classical particles
- Box of equal side length L
- Particles are non-interacting "hard spheres" and collide elastically
- Wall specularly reflects particles

### Implementation

Rather than iterating through each particle for every time-step, an event-based molecular dynamics engine jumps across time-steps until a collision occurs, and this algorithm is repeated for the duration of the simulation run. An event queue keeps track of upcoming collisions, sorted by time of occurrence.

<img src="ideal-gas-design-diagram.png">

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#Program Parameters

N = 10 #Number of particles
L = 100 #Length of box
pos = L*np.random.rand(N, 2) #Array of initial particle positions
pos += (1-pos)//(0.95*L)*(0.05*L)  #Move particles away from boundary
pos -= pos//(0.95*L)*(0.05*L)
vel = np.random.rand(N, 2) #Array of initial particle velocities
mass = np.ones(N) #Particles masses
radius = np.ones(N) #Particle radii


In [21]:
arr1 = [1,2,3,4,5]
arr2 = [6,7,8,9,0]
arr = [*zip(arr1, arr2)]


In [22]:
arr

[(1, 6), (2, 7), (3, 8), (4, 9), (5, 0)]

The **State** object controls simulation time, so it must have a method to evolve time.

The simulation time stays in the **State** object, so calculation of collision time must be facilitated by **State**.

In [ ]:
import numpy as np
from itertools import combinations

In [ ]:


class State():
    #Create State with particle number, box length, timestep, mass, and radius parameters
    #TODO: Change mass and radius parameters from constants to distributions
    #TODO: Calculate timestep in terms of precision (box length/timestep),
    #      where precision is the number of iterations required for a particle to traverse the length of the box at a speed of 1.
    def __init__(self, N, L, delta_t, M, R):
        self.N = N
        self.L = L
        self.timestep = delta_t
        self.statetime = 0.0
        #Create particles
        init_pos = L*np.random.rand(N, 2) #Array of initial particle positions
        init_pos += (1-pos)//(0.95*L)*(0.05*L)  #Move particles away from boundary
        init_pos -= pos//(0.95*L)*(0.05*L)
        init_vel = np.random.rand(N, 2)   
        self.particle_list = [Particle(init_pos[i], init_vel[i], M, R, i) for i in range(N)]
        #Create Event Queue
        event_queue = []
        particle_pairs = combinations(list(range(N)), 2)
        #Particle-Particle Collisions
        for IDs in particle_pairs:
            P1 = self.particle_list[IDs[0]]
            P2 = self.particle_list[IDs[1]]           
            event = Collision((P1, P2))   #Create Collision object
            t = collision_timestep(event)   #Calculate time until collision
            event_queue.append((event, t))
        #Particle-Wall Collisions
        for P in self.particle_list:
            wall = wall_type(P)

    def evolve_particle(self, particle, i=1):
        #Move particle
        new_pos = particle.pos + particle.vel*self.timestep*i
        #Give new position
        return new_pos

    def update_event_queue(particle_ID):
        pass

    def collision_timestep(self, event):
        if isinstance(event, Collision): 
            distance = 3*self.R   #Initialise distance to pass the while condition
            i = 1 #Iteration variable to compute motion of particles
            while(distance > 2*self.R):
                #Evolve particle by *i* timesteps               
                distance = np.abs(evolve_particle(event.P[0], i) - evolve_particle(event.P[1].pos, i))
                i++

        elif isinstance(event, BoundaryCollision): 
            pass

        return i*self.timestep #Return number of timesteps until collision
    
    def wall_type(particle):
    
    def main():
        



In [ ]:
class EventQueue():
    def __init__(self, queue)

In [24]:
#Try Object-oriented approach
class Particle():
    def __init__(self, pos, vel, mass, radius, id):
        self.pos = pos
        self.vel = vel
        self.m = mass
        self.r = radius
        self.ID = id

    def update_vel(self, vel):
        self.vel = vel

    def get_momentum(self):
        p = self.m*self.vel
        return p
           

In [ ]:
class Collision():
    def __init__(self, particles):
        self.P = particles
        #TO time OR NOT TO time?

    def collide(self):
        part1 = self.P[0]
        part2 = self.P[1]
        unit = (part1.pos-part2.pos)/np.abs(part1.pos-part2.pos)
        momentum = -2.0*part1.m*part2.m/(part1.m+part2.m)*np.dot((part1.vel-part2.vel), unit)*unit
        new_vels = [part1.vel+momentum/part1.m, part2.vel-momentum/part2.m]
        return new_vels
    
    #TODO - Collision time should be calculated in State object as it knows timestep value
    def when_collision(P):
        #Calibrate current time at zero
        t = 0.0
        #Particle positions
        pos1 = P[0].pos
        pos2 = P[1].pos

        dist = np.abs(P[0].pos - P[1].pos)
        

    
class BoundaryCollision():
    def __init__(self, particle, wall, time)
        self.P = particle
        self.N = unit_normal(wall)
        self.t = time

    def collide(self):
        vel = self.P.vel
        new_vel = vel - 2.0*np.dot(vel, self.N)*self.N
        return new_vel

    def unit_normal(wall):
        match wall:
            case "upper":
                return np.array([0,-1])
            case "left":
                return np.array([1,0])
            case "lower":
                return np.array([0,1])
            case "right":
                return np.array([-1,0])
            



In [ ]:
from itertools import combinations
particle_pairs = combinations(list(range(N)), 2)       

class EventQueue():
    def __init__(self, particle_list):
        #Cannot calculate time evolution, copy event_queue definition from state
        #Add (event, time) tuples to event queue for each pair of particles
        for pair in particle_pairs:
            P1 = particle_list[pair[0]]
            P2 = particle_list[pair[1]]
            #Collision object
            event = Collision((P1, P2))
            #Calculate time until collision


    calculate_time(event):
            